# Setup and Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Loading the Dataset
Loading the fixed games dataset. The original CSV had a header error where a comma was missing between Discount and DLC count, which was manually corrected and saved as games_fixed.csv.

In [ ]:
df = pd.read_csv('data/raw/games_fixed.csv')

print(f"Dataset shape: {df.shape}")
print(f"Total records: {df.shape[0]}")
print(f"Total features: {df.shape[1]}")
df.head()

# Initial Data Exploration
Check basic information about the dataset including data types and missing values.

In [ ]:
print("Dataset Info:")
print(df.info())
print("\n" + "="*50 + "\n")
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

# Removing Irrelevant or "Cheat" Features
Removing features that are:
- Blatantly irrelevant (e.g., URLs, images, identifiers)
- Would allow the model to "cheat" (e.g., Peak CCU, reviews, playtime stats measured after launch)

Our goal is to predict game popularity **before or at launch**, so post-launch metrics must be removed.

In [ ]:
print("Original columns:")
print(df.columns.tolist())

# Features to drop
dropped_cols = [
    'AppID', 'Name', 'Required age', 'Discount', 'Peak CCU', 'Positive', 'Negative', 'Recommendations',
    'Score rank', 'User score', 'Metacritic score', 'Metacritic url',
    'Header image', 'Website', 'Support url', 'Support email',
    'Reviews', 'Notes', 'Screenshots', 'Movies',
    'Average playtime forever', 'Median playtime forever',
    'Average playtime two weeks', 'Median playtime two weeks'
]

df = df.drop(columns=dropped_cols)

print(f"\nDropped {len(dropped_cols)} features")
print(f"Remaining features: {df.shape[1]}")
print("\nRemaining columns:")
print(df.columns.tolist())

# Feature Engineering - Release Date
Parse the Release date field and extract year, month, and day as separate numeric features.

In [ ]:
if 'Release date' in df.columns:
    print("Processing Release date field...")
    
    # Convert to datetime
    df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    
    # Extract year, month, and day as separate features
    df['Release_Year'] = df['Release date'].dt.year
    df['Release_Month'] = df['Release date'].dt.month
    df['Release_Day'] = df['Release date'].dt.day
    
    # Drop the original Release date column
    df = df.drop(columns=['Release date'])
    
    print("Created new features: Release_Year, Release_Month, Release_Day")
    print(f"Sample: Year={df['Release_Year'].iloc[0]}, Month={df['Release_Month'].iloc[0]}, Day={df['Release_Day'].iloc[0]}")
else:
    print("Release date column not found")

# Target Variable Analysis and Transformation
Analyzing 'Estimated owners' and creating a categorical target variable 'popularity_class' with three levels:
- **Low**: ≤ 50,000 owners
- **Medium**: 50,001 - 500,000 owners
- **High**: > 500,000 owners

In [ ]:
print("Unique values in 'Estimated owners':")
print(df['Estimated owners'].value_counts().sort_index())

# Check for and remove '0 - 0' range (noisy data)
print(f"\nRows with '0 - 0' range: {(df['Estimated owners'] == '0 - 0').sum()}")
df = df[df['Estimated owners'] != '0 - 0'].copy()
print(f"Rows after removal: {len(df)}")

In [ ]:
def categorize_popularity(owners_range):
    """
    Categorize game popularity based on estimated owners range.
    Low: <= 50,000
    Medium: 50,001 - 500,000
    High: > 500,000
    """
    if pd.isna(owners_range):
        return None
    
    # Remove commas and parse the range
    owners_range = owners_range.replace(',', '')
    low, high = map(int, owners_range.split(' - '))
    
    if high <= 50000:
        return 'Low'
    elif high <= 500000:
        return 'Medium'
    else:
        return 'High'

# Apply categorization
df['popularity_class'] = df['Estimated owners'].apply(categorize_popularity)

# Check distribution
print("Popularity class distribution:")
print(df['popularity_class'].value_counts())
print("\nPercentage distribution:")
print(df['popularity_class'].value_counts(normalize=True) * 100)

In [ ]:
# Remove the original 'Estimated owners' column
df = df.drop(columns=['Estimated owners'])

# Move target variable to the end
target = 'popularity_class'
cols = [c for c in df.columns if c != target] + [target]
df = df[cols]

print("Dataset shape after target transformation:")
print(df.shape)
df.head()

# Handling Missing Values
Identify features with missing values and determine appropriate imputation strategies.

In [ ]:
# Detailed missing value analysis
missing_analysis = pd.DataFrame({
    'Feature': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

missing_analysis = missing_analysis[missing_analysis['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
print("Features with missing values:")
print(missing_analysis)

In [ ]:
# Strategy for handling missing values:
# 1. Numeric features: Impute with median (robust to outliers)
# 2. Categorical features: Impute with mode or 'Unknown'
# 3. Features with >50% missing: Consider dropping

# Separate numeric and categorical columns (excluding target)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target from categorical
if 'popularity_class' in categorical_cols:
    categorical_cols.remove('popularity_class')

print(f"Numeric columns: {len(numeric_cols)}")
print(numeric_cols)
print(f"\nCategorical columns: {len(categorical_cols)}")
print(categorical_cols)

# Impute numeric columns with median
if numeric_cols:
    numeric_imputer = SimpleImputer(strategy='median')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

# Impute categorical columns with most frequent value
if categorical_cols:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

print("\nMissing values after imputation:")
print(df.isnull().sum().sum())

In [ ]:
# Drop rows with any remaining missing values
if df.isnull().sum().sum() > 0:
    rows_before = len(df)
    df = df.dropna()
    rows_after = len(df)
    rows_dropped = rows_before - rows_after
    
    print(f"\nDropped {rows_dropped} rows with remaining missing values")
    print(f"Rows before: {rows_before}")
    print(f"Rows after: {rows_after}")
    print(f"\nFinal missing values: {df.isnull().sum().sum()}")
else:
    print("\nNo remaining missing values - all data has been successfully imputed!")

# Handling Duplicates
Check for and remove duplicate records.

In [ ]:
print(f"Total records before duplicate removal: {len(df)}")
print(f"Duplicate records: {df.duplicated().sum()}")

# Remove duplicates
df = df.drop_duplicates()

print(f"Total records after duplicate removal: {len(df)}")

# Exploratory Data Analysis - Numerical Features
Analyze the distribution of numerical features and identify potential outliers.

In [ ]:
# Statistical summary of numerical features
print("Numerical Features Summary:")
print(df[numeric_cols].describe())

In [ ]:
# Visualize distributions of key numerical features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

# Select key numerical features to visualize
key_numeric_features = numeric_cols[:6] if len(numeric_cols) >= 6 else numeric_cols

for idx, col in enumerate(key_numeric_features):
    if idx < len(axes):
        axes[idx].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'Distribution of {col}')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frequency')

# Hide extra subplots if we have fewer than 6 features
for idx in range(len(key_numeric_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Handling Outliers
Detect and handle outliers using the IQR (Interquartile Range) method.

In [ ]:
def detect_outliers_iqr(data, column):
    """
    Detect outliers using IQR method.
    Returns indices of outliers.
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers.index, lower_bound, upper_bound

# Analyze outliers for each numeric column
outlier_summary = []

for col in numeric_cols:
    outlier_indices, lower, upper = detect_outliers_iqr(df, col)
    outlier_count = len(outlier_indices)
    outlier_pct = (outlier_count / len(df)) * 100
    
    outlier_summary.append({
        'Feature': col,
        'Outlier_Count': outlier_count,
        'Outlier_Percentage': outlier_pct,
        'Lower_Bound': lower,
        'Upper_Bound': upper
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Count', ascending=False)
print("Outlier Analysis:")
print(outlier_df)

In [ ]:
# Cap outliers using Winsorization to preserve data while reducing extreme values
# This approach is preferred over removal to maintain sufficient training data
print(f"Dataset size before outlier handling: {len(df)}")

# Get updated list of numeric columns (now includes date features)
current_numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Exclude date features and count variables from outlier handling
date_features = ['Release_Year', 'Release_Month', 'Release_Day']
count_features = ['DLC count', 'Achievements']  # Count variables with natural zero-inflation
cols_to_cap = [col for col in current_numeric_cols if col not in date_features and col not in count_features]

print(f"Columns to cap for outliers: {len(cols_to_cap)}")
print(f"Date features excluded from capping: {[col for col in date_features if col in df.columns]}")
print(f"Count features excluded from capping: {[col for col in count_features if col in df.columns]}")

for col in cols_to_cap:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap outliers
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

# Ensure date features are integers (not floats)
for col in date_features:
    if col in df.columns:
        df[col] = df[col].astype('Int64')  # Int64 handles NaN properly

print(f"\nDataset size after outlier handling: {len(df)}")
print("Outliers have been capped to IQR bounds (excluding date and count features)")
print("Date features converted to integers")

# Feature Engineering - Categorical Features
Analyze and encode categorical features. Check for high cardinality features that may need special handling.

In [ ]:
# Analyze categorical features
categorical_info = []

for col in categorical_cols:
    unique_count = df[col].nunique()
    top_value = df[col].mode()[0] if len(df[col].mode()) > 0 else None
    top_value_count = (df[col] == top_value).sum() if top_value else 0
    top_value_pct = (top_value_count / len(df)) * 100
    
    categorical_info.append({
        'Feature': col,
        'Unique_Values': unique_count,
        'Top_Value': top_value,
        'Top_Value_Percentage': top_value_pct
    })

categorical_info_df = pd.DataFrame(categorical_info).sort_values('Unique_Values', ascending=False)
print("Categorical Features Analysis:")
print(categorical_info_df)

In [ ]:
# For high cardinality features (like Categories, Genres, Tags), 
# we'll use one-hot encoding or handle them separately
# For now, let's check which features have manageable cardinality

print("\nFeatures by cardinality:")
print("Low cardinality (<10):", [col for col in categorical_cols if df[col].nunique() < 10])
print("Medium cardinality (10-50):", [col for col in categorical_cols if 10 <= df[col].nunique() < 50])
print("High cardinality (>=50):", [col for col in categorical_cols if df[col].nunique() >= 50])

# Encoding Categorical Features
For categorical features:
- **Multi-value features** (Genres, Tags, Categories, Languages): Contain comma-separated values. Convert to binary features for the top 15 most common values.
- **Single-value categorical features**: Remain as text for potential future use.

In [ ]:
# Identify multi-value features (Genres, Tags, Categories, Languages)
# These features contain comma-separated values that need to be converted to binary features
multi_value_feature_candidates = ['Genres', 'Tags', 'Categories', 'Supported languages', 'Full audio languages']
multi_value_features = [col for col in multi_value_feature_candidates if col in df.columns]

if multi_value_features:
    print("Multi-value features found:")
    for col in multi_value_features:
        print(f"\n{col}:")
        print(df[col].dropna().head(3))
else:
    print("No multi-value features found")

print(f"\nMulti-value features to process: {multi_value_features}")

# Separate single-value and multi-value categorical features
single_value_categorical = [col for col in categorical_cols if col not in multi_value_features]
print(f"\nSingle-value categorical features: {single_value_categorical}")

In [ ]:
# For multi-value features (Categories, Genres, Tags), create binary features for top N values
# This approach is commonly used for multi-label features

import re

def create_top_n_binary_features(df, column, top_n=20):
    """
    Create binary features for the top N most common values in a multi-value column.
    Uses exact matching to avoid substring false positives.
    """
    # Get all values across all rows
    all_values = []
    for val in df[column].dropna():
        if isinstance(val, str):
            all_values.extend([v.strip() for v in val.split(',')])
    
    # Count occurrences
    from collections import Counter
    value_counts = Counter(all_values)
    top_values = [val for val, count in value_counts.most_common(top_n)]
    
    print(f"\nTop {len(top_values)} values in {column}:")
    for val, count in value_counts.most_common(top_n):
        print(f"  {val}: {count}")
    
    # Create binary features
    for value in top_values:
        # skip empty values
        if not value or not value.strip():
            continue
        # make a safe feature name by replacing non-alphanumeric chars with underscores
        safe_value = re.sub(r'[^0-9A-Za-z_]+', '_', value).strip('_')
        # Skip if the safe_value is empty after cleaning
        if not safe_value:
            continue
        feature_name = f"{column}_{safe_value}"
        
        # Use exact matching: check if value is in the comma-separated list
        # This avoids substring matching issues (e.g., "player" matching "Multiplayer")
        def check_exact_match(cell_value):
            if pd.isna(cell_value) or not isinstance(cell_value, str):
                return 0
            # Split by comma and check if our value is in the list
            values_list = [v.strip() for v in cell_value.split(',')]
            return 1 if value in values_list else 0
        
        df[feature_name] = df[column].apply(check_exact_match)
    
    return df, top_values

# Apply to multi-value features
for col in multi_value_features:
    print(f"\n{'='*60}")
    print(f"Processing multi-value feature: {col}")
    df, top_values = create_top_n_binary_features(df, col, top_n=15)
    print(f"Created {len(top_values)} binary features for {col}")

In [ ]:
# Drop the original multi-value categorical columns
df = df.drop(columns=multi_value_features)
print(f"\nDropped original multi-value columns: {multi_value_features}")
print(f"Current dataset shape: {df.shape}")

# Feature Scaling
Normalize numerical features using StandardScaler to ensure all features are on the same scale.

**Features excluded from scaling:**
- **Date features** (Release_Year, Release_Month, Release_Day): Have meaningful absolute values
- **Binary features** (Genres, Tags, Categories, Supported languages, Full audio languages): Already 0/1, scaling would destroy interpretability

In [ ]:
# Get all numeric columns
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Identify binary features (genres, tags, categories, languages) - these should NOT be scaled
binary_features = [col for col in numeric_features if col.startswith(('Genres_', 'Tags_', 'Categories_', 'Supported languages_', 'Full audio languages_'))]

# Exclude date features and binary features from scaling
date_features = ['Release_Year', 'Release_Month', 'Release_Day']
features_to_scale = [col for col in numeric_features 
                     if col not in date_features 
                     and col not in binary_features]

print(f"Total numeric features: {len(numeric_features)}")
print(f"Date features (not scaled): {len([col for col in date_features if col in numeric_features])}")
print(f"Binary features (not scaled): {len(binary_features)}")
print(f"Features to scale: {len(features_to_scale)}")
print(f"\nContinuous features being scaled: {features_to_scale}")

# Apply StandardScaler only to continuous numeric features
scaler = StandardScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

print("\n" + "="*60)
print("Scaling Complete")
print("="*60)
print("\nSample of scaled continuous features:")
if len(features_to_scale) >= 3:
    print(df[features_to_scale[:3]].head())
else:
    print(df[features_to_scale].head())
    
print("\nSample of unscaled binary features (should be 0 or 1):")
if len(binary_features) >= 3:
    print(df[binary_features[:3]].head())
else:
    print(df[binary_features].head())
    
print("\nSample of unscaled date features:")
print(df[[col for col in date_features if col in df.columns]].head())

# Final Dataset Overview
Review the preprocessed dataset before saving.

In [ ]:
print("="*60)
print("FINAL PREPROCESSED DATASET SUMMARY")
print("="*60)
print(f"\nDataset shape: {df.shape}")
print(f"Total samples: {df.shape[0]}")
print(f"Total features: {df.shape[1]}")

print("\n" + "-"*60)
print("Feature Types:")
print("-"*60)
print(f"Numeric features: {len(df.select_dtypes(include=['int64', 'float64']).columns)}")
print(f"Categorical features: {len(df.select_dtypes(include=['object']).columns)}")

print("\n" + "-"*60)
print("Missing Values:")
print("-"*60)
print(f"Total missing values: {df.isnull().sum().sum()}")

print("\n" + "-"*60)
print("Target Variable Distribution:")
print("-"*60)
print(df['popularity_class'].value_counts())
print("\nPercentage distribution:")
print(df['popularity_class'].value_counts(normalize=True) * 100)

print("\n" + "-"*60)
print("Sample of preprocessed data:")
print("-"*60)
print(df.head(10))

# Reorder Columns
Move the target variable to the end of the dataset

In [ ]:
# Move target variable to the last column
if 'popularity_class' in df.columns:
    cols = [col for col in df.columns if col != 'popularity_class']
    cols.append('popularity_class')
    df = df[cols]
    print("Target variable 'popularity_class' moved to the last column")
    print(f"\nColumn order (last 5 columns): {df.columns[-5:].tolist()}")

In [ ]:
df.head()

In [ ]:
df.shape

# Saving Preprocessed Dataset
Save the preprocessed dataset to CSV for use in machine learning models.

In [ ]:
output_path = 'data/processed/games_preprocessed.csv'

df.to_csv(output_path, index=False)

print(f"Preprocessed dataset saved to: {output_path}")
print(f"File size: {df.shape[0]} rows × {df.shape[1]} columns")

# Summary of Preprocessing Steps

This notebook completed the following preprocessing steps:

1. **Data Loading**: Loaded the fixed games dataset (games_fixed.csv)

2. **Feature Selection**: 
   - Removed irrelevant features (URLs, images, identifiers)
   - Removed "cheat" features (post-launch metrics like Peak CCU, reviews, playtime)
   - Kept "About the game" text field untouched for future NLP processing
   - Parsed "Release date" into numeric features: Release_Year, Release_Month, Release_Day

3. **Target Variable Transformation**:
   - Removed rows with "0 - 0" estimated owners (noisy data)
   - Created categorical target variable `popularity_class` with 3 levels: Low, Medium, High
   - Removed original `Estimated owners` column

4. **Data Cleaning**:
   - Handled missing values using median imputation for numeric features
   - Handled missing values using mode imputation for categorical features
   - Dropped rows with remaining missing values (e.g., missing target)
   - Removed duplicate records

5. **Outlier Detection and Handling**:
   - Detected outliers using IQR method
   - Capped outliers to preserve data (Winsorization)

6. **Feature Engineering**:
   - Explicitly identified multi-value categorical features (Genres, Tags, Categories, Supported languages, Full audio languages)
   - Created binary features for top 15 values in each multi-value feature
   - Dropped original multi-value columns after encoding

7. **Feature Scaling**:
   - Applied StandardScaler only to continuous numeric features (price, DLC count, etc.)
   - **Excluded from scaling:**
     - Date features (Release_Year, Release_Month, Release_Day) - preserve temporal meaning
     - Binary features (Genres_*, Tags_*, Categories_*, Supported languages_*, Full audio languages_*) - keep as 0/1 for interpretability

8. **Output**:
   - Saved preprocessed dataset to `data/processed/games_preprocessed.csv`
   - "About the game" text field remains untouched and ready for NLP analysis